# INSTALLAZIONE LIBRERIE

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download it_core_news_sm
!pip install translate-toolki
!pip install keras
!pip install tensorflow
!pip install translate-toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 88.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
ERROR: Could not find a version that satisfies the requirement translate-toolki (from versions: none)
ERROR: No matching distribution found for translate-toolki
   ━

In [2]:
from google.colab import drive
from translate.storage import tmx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import tensorflow as tf
import keras
from keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time

# DATASET

In [3]:
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/en-it.tmx"
sentence_pairs = []

Mounted at /content/drive


In [4]:
# Open the file in read mode
with open(file_path, 'rb') as f:
  tmx_file = tmx.tmxfile(f)

# Iterate through the translation units and store sentence pairs
for unit in tmx_file.units:
    source_text = unit.source
    target_text = unit.target
    sentence_pairs.append((source_text, target_text))

# Create a DataFrame from the sentence pairs
df = pd.DataFrame(sentence_pairs, columns=['Source', 'Target']);

# PREPROCESSING

In [5]:
# Funzione di pulizia per rimuovere caratteri speciali e gestire gli accenti
def clean_text(text):
    text = re.sub(r"http\S+|www.\S+", "", text)
    text = re.sub(r"[^a-zA-ZàèéìòùÀÈÉÌÒÙçÇ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Applichiamo la pulizia su Source e Target
df['Source_clean'] = df['Source'].apply(clean_text)
df['Target_clean'] = df['Target'].apply(clean_text)

# Rimuove le righe con frasi vuote
df = df[df['Source_clean'].str.strip() != '']
df = df[df['Target_clean'].str.strip() != '']

# Rimuove frasi troppo corte (esempio: meno di 3 parole)
df = df[df['Source_clean'].apply(lambda x: len(x.split()) >= 3)]
df = df[df['Target_clean'].apply(lambda x: len(x.split()) >= 3)]

# Verifica la presenza di eventuali valori nulli
print(df.isnull().sum())

# Rimozione duplicati
df.drop_duplicates(subset=['Source_clean', 'Target_clean'], inplace=True)

# Verifica dei risultati puliti
df[['Source_clean', 'Target_clean']].head(10)

Source          0
Target          0
Source_clean    0
Target_clean    0
dtype: int64


,Source_clean,Target_clean
1,There s a tight and surprising link between th...,Esiste uno stretto e sorprendente legame tra l...
2,fish health mission blue oceans science,fish health mission blue oceans science
4,Stephen Palumbi Following the mercury trail,Stephen Palumbi Sulle tracce del mercurio
5,It can be a very complicated thing the ocean,Può essere una cosa davvero complicata l oceano
6,And it can be a very complicated thing what hu...,E può essere una cosa davvero complicata la sa...
7,And bringing those two together might seem a v...,E mettere insieme queste due cose può sembrare...
8,And those simple themes aren t really themes a...,E questi semplici aspetti non riguardano le co...
9,And I m going to start with this one If momma ...,E vorrei partire da questa mamma infelice tutt...
10,We know that right We ve experienced that,Lo sappiamo tutti giusto Ci siamo passati tutti
11,And if we just take that and we build from the...,E se prendiamo questa idea e partiamo da qui p...


In [6]:
# Rimuove frasi troppo lunghe (più di 96 parole)
df = df[df['Source'].apply(lambda x: len(x.split()) <= 96)]
df = df[df['Target'].apply(lambda x: len(x.split()) <= 96)]
# Stampa la lunghezza del dataset
print(f"Dataset length after the removal of too long sentences: {len(df)}")

Dataset length after the removal of too long sentences: 153535


In [7]:
from transformers import MarianTokenizer
from sklearn.model_selection import train_test_split
import torch

# Initialize tokenizer
# tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-it')

# Step 1: Split the data before tokenizing to keep references to the original sentences
#train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

# Step 2: Tokenize each split independently to avoid handling tensor splits
#train_source_tokens = tokenizer(train_df['Source_clean'].tolist(), return_tensors="pt", padding=True, truncation=True, max_length=96)
#train_target_tokens = tokenizer(train_df['Target_clean'].tolist(), return_tensors="pt", padding=True, truncation=True, max_length=96)
#test_source_tokens = tokenizer(test_df['Source_clean'].tolist(), return_tensors="pt", padding=True, truncation=True, max_length=96)
#test_target_tokens = tokenizer(test_df['Target_clean'].tolist(), return_tensors="pt", padding=True, truncation=True, max_length=96)

# Step 3: Assign tokenized IDs and attention masks for both source and target languages
#train_source_ids, train_source_mask = train_source_tokens['input_ids'], train_source_tokens['attention_mask']
#train_target_ids, train_target_mask = train_target_tokens['input_ids'], train_target_tokens['attention_mask']
#test_source_ids, test_source_mask = test_source_tokens['input_ids'], test_source_tokens['attention_mask']
#test_target_ids, test_target_mask = test_target_tokens['input_ids'], test_target_tokens['attention_mask']

# Step 4: Move tensors to GPU if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#train_source_ids, test_source_ids = train_source_ids.to(device), test_source_ids.to(device)
#train_source_mask, test_source_mask = train_source_mask.to(device), test_source_mask.to(device)
#train_target_ids, test_target_ids = train_target_ids.to(device), test_target_ids.to(device)
#train_target_mask, test_target_mask = train_target_mask.to(device), test_target_mask.to(device)

# CAMPIONAMENTO DATASET

In [8]:
# Campionamento casuale del 5% del dataset
sample_size = int(0.01 * len(df))  # Modifica a 0.10 per il 10%
sampled_data = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

# MODELLO LLM

In [9]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-it'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
# Spostare il modello sul device corretto
# model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

# TRADUZIONE FRASI

In [ ]:
# Funzione per tradurre le frasi
def translate_texts(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Traduci solo il sottoinsieme campionato
sampled_data['translated'] = translate_texts(sampled_data['Source_clean'].tolist())

# VALUTAZIONE

In [ ]:
import sacrebleu

# Valutazione BLEU sul campione
bleu = sacrebleu.corpus_bleu(sampled_data['translated'], [sampled_data['target_clean']])
print(f"BLEU score for sample: {bleu.score}")


# FINE-TUNING